In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# ===== CONFIGURAÇÕES =====
input_path = '/Users/marcelosilva/Desktop/projectOne/4/F-TargetLabelEnconding/Test Dataset/DataSetTarget.csv'
output_dir = '/Users/marcelosilva/Desktop/projectOne/5/A-TrainTestDataset'

print("🚀 DIVIDINDO DATASET EM TRAIN (85%) E TEST (15%)")
print(f"📁 Dataset origem: {input_path}")

# ===== CARREGAR DATASET =====
try:
    df = pd.read_csv(input_path)
    print(f"✅ Dataset carregado!")
    print(f"📊 Shape: {df.shape}")
except FileNotFoundError:
    print(f"❌ ERRO: Arquivo não encontrado!")
    exit()

# ===== VERIFICAR DISTRIBUIÇÃO DO TARGET =====
print("\n🎯 DISTRIBUIÇÃO DO TARGET:")
target_counts = df['status_nutricional_who'].value_counts().sort_index()
print("Distribuição atual:")
for value, count in target_counts.items():
    percentage = (count / len(df)) * 100
    print(f"  Classe {value}: {count} ({percentage:.1f}%)")

# Verificar se tem a classe 0 (Desnutrido)
if 0 not in target_counts.index:
    print("❌ ERRO: Classe 0 (Desnutrido) não encontrada!")
    exit()

print(f"\n✅ Classe 0 (Desnutrido): {target_counts[0]} casos ({(target_counts[0]/len(df))*100:.1f}%)")

# ===== DIVISÃO ESTRATIFICADA =====
print(f"\n📊 FAZENDO SPLIT 85% TRAIN / 15% TEST:")

X = df.drop(['status_nutricional_who'], axis=1)
y = df['status_nutricional_who']

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.15,
    train_size=0.85,
    stratify=y,
    random_state=123
)

# Recriar DataFrames
train_df = pd.concat([X_train, y_train], axis=1)
test_df = pd.concat([X_test, y_test], axis=1)

print(f"✅ TrainDS: {train_df.shape}")
print(f"✅ TestDS: {test_df.shape}")

# ===== VERIFICAR ESTRATIFICAÇÃO =====
print(f"\n🔍 VERIFICANDO ESTRATIFICAÇÃO:")

print("TRAIN dataset:")
train_counts = train_df['status_nutricional_who'].value_counts().sort_index()
for value, count in train_counts.items():
    percentage = (count / len(train_df)) * 100
    print(f"  Classe {value}: {count} ({percentage:.1f}%)")

print("\nTEST dataset:")
test_counts = test_df['status_nutricional_who'].value_counts().sort_index()
for value, count in test_counts.items():
    percentage = (count / len(test_df)) * 100
    print(f"  Classe {value}: {count} ({percentage:.1f}%)")

# ===== VERIFICAR CLASSE 0 (DESNUTRIDO) =====
print(f"\n⚠️  CLASSE MINORITÁRIA (0 - Desnutrido):")
train_class0 = (y_train == 0).sum()
test_class0 = (y_test == 0).sum()
total_class0 = (y == 0).sum()

print(f"Total: {total_class0}")
print(f"Train: {train_class0} ({(train_class0/total_class0)*100:.1f}%)")
print(f"Test: {test_class0} ({(test_class0/total_class0)*100:.1f}%)")

if train_class0 > 0 and test_class0 > 0:
    print("✅ Classe 0 presente em ambos conjuntos!")
else:
    print("❌ ERRO: Classe 0 ausente em algum conjunto!")

# ===== CRIAR DIRETÓRIO E SALVAR =====
print(f"\n📁 Criando diretório: {output_dir}")
os.makedirs(output_dir, exist_ok=True)

train_path = os.path.join(output_dir, 'TrainDS.csv')
test_path = os.path.join(output_dir, 'TestDS.csv')

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)

print(f"✅ TrainDS salvo: {train_path}")
print(f"✅ TestDS salvo: {test_path}")

# ===== RESUMO FINAL =====
print(f"\n{'='*50}")
print("📋 RESUMO DO SPLIT")
print(f"{'='*50}")
print(f"📊 Dataset original: {df.shape}")
print(f"📊 TrainDS (85%): {train_df.shape}")
print(f"📊 TestDS (15%): {test_df.shape}")
print(f"🎯 Estratificação mantida para todas as 4 classes")
print(f"✅ Classe 0 (Desnutrido) preservada nos dois conjuntos")
print(f"🚀 Pronto para PyCaret!")